# SELECT Part 1

We'll explore different types of queries that can be performed with the SELECT statement.

In [1]:
import sqlite3
db = sqlite3.connect('example.db')

### Select *

In [2]:
cursor = db.execute('SELECT * FROM country;')
cursor.fetchall()

[(1, 'United Kingdom'), (2, 'USA'), (3, 'Republic of Ireland')]

In [3]:
cursor = db.execute('SELECT * FROM book;')
cursor.fetchall()

[(1, 3, '1984', 'XYZ-1'),
 (2, 2, 'The Happy Prince', 'XYZ-2'),
 (3, 2, 'The Picture of Dorian Gray', 'XYZ-3'),
 (4, 1, 'The Adventures of Tom Sawyer', 'XYZ-4'),
 (5, 1, 'The Adventures of Huckleberry Finn', 'XYZ-5'),
 (6, 2, 'The Canterville Ghost', 'XYZ-6'),
 (7, 3, 'Animal Farm', 'XYZ-7')]

In [4]:
cursor = db.execute('SELECT * FROM book;')
for row in cursor:
    id = row[0]
    title = row[2]
    print("{} - {}".format(id, title))

1 - 1984
2 - The Happy Prince
3 - The Picture of Dorian Gray
4 - The Adventures of Tom Sawyer
5 - The Adventures of Huckleberry Finn
6 - The Canterville Ghost
7 - Animal Farm


### Select columns

In [5]:
cursor = db.execute('SELECT id, name FROM country;')
cursor.fetchall()

[(1, 'United Kingdom'), (2, 'USA'), (3, 'Republic of Ireland')]

In [6]:
cursor = db.execute('SELECT id, title FROM book;')
cursor.fetchall()

[(1, '1984'),
 (2, 'The Happy Prince'),
 (3, 'The Picture of Dorian Gray'),
 (4, 'The Adventures of Tom Sawyer'),
 (5, 'The Adventures of Huckleberry Finn'),
 (6, 'The Canterville Ghost'),
 (7, 'Animal Farm')]

In [7]:
cursor = db.execute('SELECT id, title FROM book;')
for row in cursor:
    id = row[0]
    title = row[1]
    print("{} - {}".format(id, title))

1 - 1984
2 - The Happy Prince
3 - The Picture of Dorian Gray
4 - The Adventures of Tom Sawyer
5 - The Adventures of Huckleberry Finn
6 - The Canterville Ghost
7 - Animal Farm


### Select Columns, accessed by names

In [8]:
db.row_factory = sqlite3.Row

In [9]:
cursor = db.execute('SELECT id, title FROM book;')
cursor.fetchall()

In [10]:
cursor = db.execute('SELECT id, title FROM book;')
row = cursor.fetchone()

In [11]:
row

In [12]:
row.keys()

['id', 'title']

In [13]:
row['title']

'1984'

In [14]:
cursor = db.execute('SELECT id, title FROM book;')
for row in cursor:
    id = row['id']
    title = row['title']
    print("{} - {}".format(id, title))

1 - 1984
2 - The Happy Prince
3 - The Picture of Dorian Gray
4 - The Adventures of Tom Sawyer
5 - The Adventures of Huckleberry Finn
6 - The Canterville Ghost
7 - Animal Farm


Using query aliases:

In [15]:
cursor = db.execute('SELECT id idx, title book_title FROM book;')
for row in cursor:
    id = row['idx']
    title = row['book_title']
    print("{} - {}".format(id, title))

1 - 1984
2 - The Happy Prince
3 - The Picture of Dorian Gray
4 - The Adventures of Tom Sawyer
5 - The Adventures of Huckleberry Finn
6 - The Canterville Ghost
7 - Animal Farm


I'll reset the Row Factory to see the results as tuples again::

In [16]:
db.row_factory = None

### Select Where

Lets you filter your results with conditions. For example, "look up all the books written by British authors":

In [17]:
cursor = db.execute('SELECT * FROM author WHERE country_id = 1')
cursor.fetchall()

[(3, 1, 'George Orwell')]

But what happens if those conditions need to be generated dynamically? For example, the country to use as filter is decided by the user through our website:

In [18]:
country_id = 1

In [19]:
cursor = db.execute('SELECT * FROM author WHERE country_id = {}'.format(country_id))

In [20]:
cursor.fetchall()

[(3, 1, 'George Orwell')]

It seems to work, but this is actually **REALLY DANGEROUS**. We're vulnerable to SQL Injection. What happens if the user, instead of setting `countr_id` to `1`, does something like:

In [21]:
country_id = '1 or 1=1; --'

In [22]:
cursor = db.execute('SELECT * FROM author WHERE country_id = {}'.format(country_id))
cursor.fetchall()

[(1, 2, 'Mark Twain'), (2, 3, 'Oscar Wilde'), (3, 1, 'George Orwell')]

We've just been subject of SQL Injection. The actual country_id can be something like:

`1 or 1=1; DELETE FROM User; --`

And we've just lost our entire `User` table.

### SQLite Parameter Substitution

Instead of using regular String formatting, we can use SQLite that will take care of escaping any malicious input. Using the previous example:

In [23]:
country_id = 1

In [24]:
params = [country_id]
cursor = db.execute('SELECT * FROM author WHERE country_id = ?', params)
cursor.fetchall()

[(3, 1, 'George Orwell')]

Parameters must be provided with a sequence, respecting the order of the placeholders (`?`):

In [25]:
cursor = db.execute('SELECT * FROM book')
cursor.fetchall()

[(1, 3, '1984', 'XYZ-1'),
 (2, 2, 'The Happy Prince', 'XYZ-2'),
 (3, 2, 'The Picture of Dorian Gray', 'XYZ-3'),
 (4, 1, 'The Adventures of Tom Sawyer', 'XYZ-4'),
 (5, 1, 'The Adventures of Huckleberry Finn', 'XYZ-5'),
 (6, 2, 'The Canterville Ghost', 'XYZ-6'),
 (7, 3, 'Animal Farm', 'XYZ-7')]

In [26]:
params = ['The%']
cursor = db.execute('SELECT * FROM book WHERE title LIKE ?', params)
cursor.fetchall()

[(2, 2, 'The Happy Prince', 'XYZ-2'),
 (3, 2, 'The Picture of Dorian Gray', 'XYZ-3'),
 (4, 1, 'The Adventures of Tom Sawyer', 'XYZ-4'),
 (5, 1, 'The Adventures of Huckleberry Finn', 'XYZ-5'),
 (6, 2, 'The Canterville Ghost', 'XYZ-6')]

In [27]:
params = ['The%', 1]
cursor = db.execute('SELECT * FROM book WHERE title LIKE ? AND author_id = ?', params)
cursor.fetchall()

[(4, 1, 'The Adventures of Tom Sawyer', 'XYZ-4'),
 (5, 1, 'The Adventures of Huckleberry Finn', 'XYZ-5')]

Sometimes, using sequences to order placeholders `?` might be confusing, so we can use "named" parameters instead:

In [28]:
params = {
    'title': 'The%',
    'author': 1
}
cursor = db.execute('SELECT * FROM book WHERE title LIKE :title AND author_id = :author', params)
cursor.fetchall()

[(4, 1, 'The Adventures of Tom Sawyer', 'XYZ-4'),
 (5, 1, 'The Adventures of Huckleberry Finn', 'XYZ-5')]